# Customize a flavor

Sometimes you need very specific dependencies or versions of dependencies in the Exasol UDFs. In such case you can customize a Script-Language Container.
You find additional information in the [Exasol official documentation](https://docs.exasol.com/db/latest/database_concepts/udf_scripts/adding_new_packages_script_languages.htm#).

## Setup
### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Instantiate SLCT Manager

The "Script-Languages-Container-Tools" Manager (SLCT Manager) simplifies using the API of `exaslct`.
The following cell will therefore create an instance class `SlctManager` from the notebook-connector.

In [ ]:
from exasol.nb_connector import slct_manager
slctmanager = slct_manager.SlctManager(ai_lab_config)

### Import some utility functions

In [ ]:
%run ./utils/file_system_ui.ipynb
%run ./utils/slc_ui.ipynb

## Customize

First you need to define an alias for the new SLC. The alias will be used to reference the container later from the UDFs.

Note: In this tutorial the alias also will be used as part of the export file (tar.gz) and the uploaded container to the BucketFS. This allows you to create, upload and use different script-language-containers: one per alias.

In [ ]:
display(get_alias_ui(ai_lab_config, "ai_lab_default_custom"))

### Flavor Customization Build Step

`exasclt` consists of multiple build steps. By a build step here we mean a file structure which serves as an input for a certain stage of the building process of the script-languages-container. See [Advanced Topics](./advanced.ipynb) for more details.

Build step **flavor_customization** is defined by a Dockerfile and several package lists. We recommend to add new packages to the package lists and only modify the Dockerfile if you need very specific changes, like adding additional resources.

In [ ]:
show_directory_content(slctmanager.slc_dir.flavor_dir / "flavor_customization")

The Dockerfile consists of two parts. The first part installs the packages from the package lists and should only be changed with care. The second part is free for your changes. Read the description in the Dockerfile carefully to find out what you can and shouldn't do.

#### Package Lists
The package lists have a unified format. Each line consists of the package name and the package version separated by the pipe character `|`, e.g `xgboost|1.3.3`. You can comment out a whole line by adding a hash character `#` the beginning of the line. You can also add a trailing comment to a package definition by adding `#` after the package definition. We usually recommend to install a specific package version to avoid surprises about which version actually gets installed.

In [ ]:
show_files([slctmanager.slc_dir.custom_pip_file])

We are now going to append Python package "xgboost" to one of the package lists by adding `xgboost|2.0.3` and `scikit-learn|1.5.0` to file `flavor_customization/packages/python3_pip_packages`. 
Notes:
 - running the following command multiple times will iteratively append the packages
 - you can also click on the link and modify the file directly

In [ ]:
xgboost_pkg = slct_manager.PipPackageDefinition(pkg="xgboost", version="2.0.3")
scikit_learn_pkg = slct_manager.PipPackageDefinition(pkg="scikit-learn", version="1.5.0")
slctmanager.append_custom_packages([xgboost_pkg, scikit_learn_pkg])
show_files([slctmanager.slc_dir.custom_pip_file])

#### Rebuilding the customized Flavor

After changing the flavor you need to rebuild it. You can do it by running `export` again. Exaslct automatically recognizes that the flavor has changed and builds a new version of the container. Don't get confused by the warnings: `exaslct` first tries to find the cached docker images (see [Advanced Topics](./advanced.ipynb)), but as the content has changed, the cached image is not available, and the docker service returns a 404 error message.

In [ ]:
slctmanager.export()

Lets check the resulting tar gz file:

In [ ]:
show_directory_content(slctmanager.working_path.export_path)

#### Upload the Container to the Database
To use the new container you need to upload it to the BucketFS. If the build machine has access to the BucketFS you do it with the `exaslct` upload command, as shown below. Otherwise you need to install the script-languages-container manually: 
1. Transfer the container tar gz file of the previous step to a machine that has access to the BucketFS. 
2. From that machine upload it via curl, as described in our [documentation](https://docs.exasol.com/db/latest/database_concepts/udf_scripts/adding_new_packages_script_languages.htm).

With the following command you upload the new script language container.
You could run the same on the command line with the `exaslct` tool:
```
cd <slc_dir>
./exaslct upload --flavor-path flavors/<flavor_path> --database-host <database_host> --bucketfs-port <bucketfs_port> --bucketfs-username <bucketfs_username> --bucketfs-password <bucketfs_password> --bucketfs-name <bucketfs_name> --bucket-name <bucket_name>
```



In [ ]:
slctmanager.upload()

This command also stores the activation statement in the ai-lab-config. You can verify it with:


In [ ]:
slctmanager.activation_key

The syntax of the activation statement is: `alias=url`. The activation key will be used in the `ALTER_SESSION` or `ALTER_SYSTEM` commands to "register" the script-language-container for usage in the UDFs.

You can generate the SQL commands for the activation with the following command line:
```
cd <slc_dir>
./exaslct generate-language-activation --flavor-path flavors/<flavor_path> --bucketfs-name <bucketfs_name> --bucket-name <bucket_name> --container-name <container_name> --path-in-bucket <path_in_bucket>
```

You can now continue [testing the uploaded container](./test_slc.ipynb).